# 🌲🌲 Vegetation experiment 🌲🌲

### Originally created by Dr. Narender Reddy and Prof. Pier Luigi Vidale (NCAS, University of Reading)

## 🌎 🌳 Vegetation and its importance on land surafce
Vegetataion has large impact on the Earth System climate. The vegetation can affect the climate by: 

- 🌞 impacting the albedo of the surface

- 🌡️ altering the soil temperatures

- 💧 altering the soil moisture

- ⚡💦 altering the water and energy budget

## 🎯 Objective
To understand the impact of the largest vegetation on earth (the amazon forest) on various large scale earth and land surface processes.

## 🐪🏝️🧪 Designing the experiment
- What do we need to change to understand the impact of Amazon on the planet?

🔸 🔄 Swapping Amazon and Sahara

- What should we replace the Amamzon with, to extract the extreme impact of the Amazon forest?

🔸 🌞🐪 Two Saharas

🔸 🌧️🏝️ Two Amazones


## 🧠 💪 Hypothesise the impacts of removing the Amazon or replacing the forest with a different land type
- Formulate testable hypotheses and evaluate them with simple diagnostics (using SpeedyWeather output).

## 🔬🧠 Guiding Questions

**⚡⚖️ Energy balance:** 
- How does replacing forest with desert alter albedo and Bowen ratio? Where does the “lost/gained” energy go?

**💧 Moisture recycling:** 
- What fraction of Amazon rainfall is recycled locally vs advected? How would making the Sahara forested change recycling efficiency?

**➿📈 Non‑linearities:** 
- Do “Two Saharas” and “Two Amazons” produce responses that are symmetric opposites? If not, why?

In [3]:
using SpeedyWeather
using GeoMakie, CairoMakie
using ProgressMeter
using IJulia

### CTRL Experiment
    - Using the climatological vegetation and albedo for running the SpeedyWeather

In [5]:
spectral_grid = SpectralGrid(trunc=31, nlayers=8)

output = NetCDFOutput(spectral_grid, PrimitiveWetModel, path="./", id="Veg_CTRL_Exp", output_dt=Hour(6))

vertical_advection = WENOVerticalAdvection(spectral_grid)

# Setting the climatological albedo
albedo = Albedo(ocean=OceanSeaIceAlbedo(spectral_grid), land=AlbedoClimatology(spectral_grid))

## Continuation run
# initial_conditions = StartFromFile(path="./", id="Veg_CTRL_Spinup",run_number=2)   # or specify `path` kwarg if not in this folder

#model = PrimitiveWetModel(spectral_grid; vertical_advection, initial_conditions, albedo, output)
model = PrimitiveWetModel(spectral_grid; vertical_advection, albedo, output)
#add!(model, SpeedyWeather.AllOutputVariables()...)
#
#model = PrimitiveWetModel(spectral_grid; vertical_advection, albedo, output)
simulation = initialize!(model)

# first arg: what you want to initialize, 2nd: always model (read only)
#initialize!(model.land, model)   # first arg: what you want to initialize, 2nd: always model (read only)

# output both SST and sea ice concentration
add!(model, SpeedyWeather.OceanOutput()...)
add!(model, SpeedyWeather.LandOutput()...)
add!(model, SpeedyWeather.RadiationOutput()...)
add!(model, SpeedyWeather.PrecipitationOutput()...)
add!(model, SpeedyWeather.SurfaceFluxesOutput()...)
add!(model, SpeedyWeather.AlbedoOutput())

# Finally, let us run the model
run!(simulation, period=Year(1), output=true)

Weather is speedy: run_Veg_CTRL_Exp_0001 100% Time: 0:02:46 (518.19 years/day)


Simulation{PrimitiveWetModel}
├ prognostic_variables::PrognosticVariables{...}
├ diagnostic_variables::DiagnosticVariables{...}
└ model::PrimitiveWetModel{...}

### 🔄 Exp. I - Swapping Amazon and Sahara
    - Swapping Amazon with Sahara (by changing the input vegetation and albedo files)

In [6]:
output_2 = NetCDFOutput(spectral_grid, PrimitiveWetModel, path="./", id="Veg_Swap_Exp", output_dt=Hour(6))

land_albedo_2 = AlbedoClimatology(spectral_grid; path="./", file="albedo_swap_exp.nc")
albedo_2 = Albedo(ocean=OceanSeaIceAlbedo(spectral_grid), land=land_albedo_2)

vegetation_2 = VegetationClimatology(spectral_grid; path="./", file="vegetation_swap_exp.nc")
land_2 = LandModel(spectral_grid; vegetation = vegetation_2)

## Continuation run
#initial_conditions_2 = StartFromFile(path="/gws/nopw/j04/hrcm/naren/SpeedyWeather", id="Veg_Swap_Spinup",run_number=2)   # or specify `path` kwarg if not in this folder

#model_2 = PrimitiveWetModel(spectral_grid; vertical_advection, initial_conditions = initial_conditions_2 , land = land_2, albedo = albedo_2, output = output_2)
model_2 = PrimitiveWetModel(spectral_grid; vertical_advection, land = land_2, albedo = albedo_2, output = output_2)

simulation_2 = initialize!(model_2)

# now offset via set! with add=true
set!(albedo_2.land, (λ, φ) -> 0.1, add=true)

# output both SST and sea ice concentration
add!(model_2, SpeedyWeather.OceanOutput()...)
add!(model_2, SpeedyWeather.LandOutput()...)
add!(model_2, SpeedyWeather.RadiationOutput()...)
add!(model_2, SpeedyWeather.PrecipitationOutput()...)
add!(model_2, SpeedyWeather.SurfaceFluxesOutput()...)
add!(model_2, SpeedyWeather.AlbedoOutput())

#initialize!(model_2.land, model_2)

# Finally, let us run the model
run!(simulation_2, period=Year(1), output=true)

Weather is speedy: run_Veg_Swap_Exp_0001 100% Time: 0:02:48 (514.10 years/day)


Simulation{PrimitiveWetModel}
├ prognostic_variables::PrognosticVariables{...}
├ diagnostic_variables::DiagnosticVariables{...}
└ model::PrimitiveWetModel{...}

### 🌞🐪 Exp. II - Two Saharas
    - Removing Amazon and replacing it with a desert (In effect earth will have two Saharas)

In [7]:
output_3 = NetCDFOutput(spectral_grid, PrimitiveWetModel, path="./", id="Veg_Sahara2_Exp", output_dt=Hour(6))

land_albedo_3 = AlbedoClimatology(spectral_grid; path="./", file="albedo_sahara2_exp.nc")
albedo_3 = Albedo(ocean=OceanSeaIceAlbedo(spectral_grid), land = land_albedo_3)

vegetation_3 = VegetationClimatology(spectral_grid; path="./", file="vegetation_sahara2_exp.nc")
land_3 = LandModel(spectral_grid; vegetation = vegetation_3)

## Continuation run
#initial_conditions_3 = StartFromFile(path="/gws/nopw/j04/hrcm/naren/SpeedyWeather", id="Veg_Sahara2_Spinup",run_number=2)   # or specify `path` kwarg if not in this folder

#model_3 = PrimitiveWetModel(spectral_grid; vertical_advection, initial_conditions = initial_conditions_3, land = land_3, albedo = albedo_3, output = output_3)
model_3 = PrimitiveWetModel(spectral_grid; vertical_advection, land = land_3, albedo = albedo_3, output = output_3)

#model_3 = PrimitiveWetModel(spectral_grid; vertical_advection, land = land_3, albedo = albedo_3, output = output_3)
simulation_3 = initialize!(model_3)

# now offset via set! with add=true
set!(albedo_3.land, (λ, φ) -> 0.1, add=true)

# output both SST and sea ice concentration
add!(model_3, SpeedyWeather.OceanOutput()...)
add!(model_3, SpeedyWeather.LandOutput()...)
add!(model_3, SpeedyWeather.RadiationOutput()...)
add!(model_3, SpeedyWeather.PrecipitationOutput()...)
add!(model_3, SpeedyWeather.SurfaceFluxesOutput()...)
add!(model_3, SpeedyWeather.AlbedoOutput())

run!(simulation_3, period=Year(15), output=true)

Weather is speedy: run_Veg_Sahara2_Exp_0001 100% Time: 1:08:43 (314.26 years/day)


Simulation{PrimitiveWetModel}
├ prognostic_variables::PrognosticVariables{...}
├ diagnostic_variables::DiagnosticVariables{...}
└ model::PrimitiveWetModel{...}

### 🌧️🏝️ Exp. III - Two Amazons
    - Replacing the largest desert (Sahara) with the Amazon.  

In [8]:
output_4 = NetCDFOutput(spectral_grid, PrimitiveWetModel, path="./", id="Veg_Amazon2_Exp", output_dt=Hour(6))

land_albedo_4 = AlbedoClimatology(spectral_grid; path="./", file="albedo_amazon2_exp.nc")
albedo_4 = Albedo(ocean=OceanSeaIceAlbedo(spectral_grid), land = land_albedo_4)

vegetation_4 = VegetationClimatology(spectral_grid; path="./", file="vegetation_amazon2_exp.nc")
land_4 = LandModel(spectral_grid; vegetation = vegetation_4)

## Continuation run
#initial_conditions_4 = StartFromFile(path="/gws/nopw/j04/hrcm/naren/SpeedyWeather", id="Veg_Amazon2_Spinup",run_number=3)   # or specify `path` kwarg if not in this folder

#model_4 = PrimitiveWetModel(spectral_grid; vertical_advection, initial_conditions = initial_conditions_4 , land = land_4, albedo = albedo_4, output = output_4)
model_4 = PrimitiveWetModel(spectral_grid; vertical_advection, land = land_4, albedo = albedo_4, output = output_4)

#model_4 = PrimitiveWetModel(spectral_grid; vertical_advection, land = land_4, albedo = albedo_4, output = output_4)
simulation_4 = initialize!(model_4)

# now offset via set! with add=true
set!(albedo_4.land, (λ, φ) -> 0.1, add=true)

# output both SST and sea ice concentration
add!(model_4, SpeedyWeather.OceanOutput()...)
add!(model_4, SpeedyWeather.LandOutput()...)
add!(model_4, SpeedyWeather.RadiationOutput()...)
add!(model_4, SpeedyWeather.PrecipitationOutput()...)
add!(model_4, SpeedyWeather.SurfaceFluxesOutput()...)
add!(model_4, SpeedyWeather.AlbedoOutput())

#simulation_4.model.land.vegetation
run!(simulation_4, period=Year(15), output=true)

Weather is speedy: run_Veg_Amazon2_Exp_0001 100% Time: 0:44:28 (485.71 years/day)


Simulation{PrimitiveWetModel}
├ prognostic_variables::PrognosticVariables{...}
├ diagnostic_variables::DiagnosticVariables{...}
└ model::PrimitiveWetModel{...}

## 🤔❓ How does the hypothesis hold at the end of the experiments?

- Are the resuls inline with your hypothesis? If Yes, what are the processes driving it? If No, what processess are missing in the current model?
